In [3]:
str(Path(os.path.abspath('')).absolute().parent.parent.parent)

'/Users/cabesd/Projects/NFLUX/Control-Tasks/shiva'

In [1]:
import sys, os
from pathlib import Path
sys.path.append(str(Path(os.path.abspath('')).absolute().parent.parent.parent))

In [2]:
import random
import numpy as np
import grpc

from shiva.core.communication_objects.env_command_pb2 import EnvironmentCommand
from shiva.core.communication_objects.env_step_pb2 import ( EnvStepInput, EnvStepOutput )
from shiva.core.communication_objects.env_specs_pb2 import EnvironmentSpecs
from shiva.core.communication_objects.env_metrics_pb2 import AgentMetrics
from shiva.core.communication_objects.agent_state_pb2 import AgentState
from shiva.core.communication_objects.service_env_pb2_grpc import EnvironmentStub, EnvironmentServicer, add_EnvironmentServicer_to_server

ModuleNotFoundError: No module named 'shiva.core.communication_objects'

In [3]:
def run():
    channel = grpc.insecure_channel('localhost:50051')
    stub = EnvironmentStub(channel)

    env_in = EnvStepInput()

    action = env_in.agent_actions['0'].data.add()
    action.data.extend(np.random.rand(5))

    action = env_in.agent_actions['1'].data.add()
    action.data.extend([np.random.randint(1,5) for _ in range(3)])

    env_in.command = EnvironmentCommand.STEP

    env_state = stub.Step(env_in)
    
    return env_state, channel, stub

In [17]:
run()

(agent_states {
   key: "0"
   value {
     data {
       next_observation {
         data: 0.6894981861114502
         data: 0.383246511220932
         data: 0.2759014368057251
         data: 0.9217872023582458
         data: 0.8580998778343201
       }
       reward: 0.6816263198852539
     }
   }
 }
 agent_states {
   key: "1"
   value {
     data {
       next_observation {
         data: 0.1677057296037674
         data: 0.3641129732131958
         data: 0.805634617805481
         data: 0.26316797733306885
         data: 0.16621264815330505
       }
       reward: 0.27741938829421997
       done: true
     }
   }
 }
 agent_metrics {
   key: "0"
   value {
     data {
       steps_per_episode: 1.0
       step_count: 2.0
       temp_done_counter: 3.0
       done_count: 4.0
       reward_per_step: 5.0
       reward_per_episode: 6.0
       reward_total: 7.0
     }
   }
 },
 <shiva.core.communication_objects.service_env_pb2_grpc.EnvironmentStub at 0x119f60f28>)

In [4]:
s = []
for _ in range(5):
    state, channel, stub = run()
    s.append(state)

In [6]:
s

[agent_states {
   key: "0"
   value {
     data {
       next_observation {
         data: 0.8854866027832031
         data: 0.3215821087360382
         data: 0.2256254106760025
         data: 0.35417839884757996
         data: 0.5626516938209534
       }
       reward: 0.7766212224960327
     }
   }
 }
 agent_states {
   key: "1"
   value {
     data {
       next_observation {
         data: 0.1212611272931099
         data: 0.8806151151657104
         data: 0.7047242522239685
         data: 0.9671302437782288
         data: 0.6195788383483887
       }
       reward: 0.8038033843040466
       done: true
     }
   }
 }
 agent_metrics {
   key: "0"
   value {
     data {
       steps_per_episode: 1.0
       step_count: 2.0
       temp_done_counter: 3.0
       done_count: 4.0
       reward_per_step: 5.0
       reward_per_episode: 6.0
       reward_total: 7.0
     }
   }
 }, agent_states {
   key: "0"
   value {
     data {
       next_observation {
         data: 0.03614870831370354
  

# Client side

In [87]:
env_in = EnvStepInput()

action = env_in.agent_actions['0'].data.add()
action.data.extend(np.random.rand(5))

action = env_in.agent_actions['0'].data.add()
action.data.extend([np.random.randint(1,5) for _ in range(5)])

env_in.command = EnvironmentCommand.STEP
env_in

agent_actions {
  key: "0"
  value {
    data {
      data: 0.6070301532745361
      data: 0.2676883637905121
      data: 0.9742514491081238
      data: 0.7766292095184326
      data: 0.10990528017282486
    }
    data {
      data: 2.0
      data: 3.0
      data: 2.0
      data: 4.0
      data: 3.0
    }
  }
}

In [88]:
v = list(env_in.agent_actions.keys())
v

['0']

In [91]:
env_in.agent_actions[v[0]].data

[data: 0.6070301532745361
data: 0.2676883637905121
data: 0.9742514491081238
data: 0.7766292095184326
data: 0.10990528017282486
, data: 2.0
data: 3.0
data: 2.0
data: 4.0
data: 3.0
]

In [95]:
agent_action = env_in.agent_actions[v[0]].data

In [98]:
len(agent_action)

2

In [96]:
agent_action[0].data

[0.6070301532745361, 0.2676883637905121, 0.9742514491081238, 0.7766292095184326, 0.10990528017282486]

In [97]:
agent_action[1].data

[2.0, 3.0, 2.0, 4.0, 3.0]

In [104]:
all_agents_instances_actions = []

In [106]:
all_agents_instances_actions.append([ agent_action[i].data for i in range(len(agent_action)) ])

In [108]:
all_agents_instances_actions[0]

[[0.6070301532745361, 0.2676883637905121, 0.9742514491081238, 0.7766292095184326, 0.10990528017282486],
 [2.0, 3.0, 2.0, 4.0, 3.0]]

# Server side

In [28]:
a = AgentState()
a.next_observation.data.extend(np.random.rand(5)) 
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0,2))
a

next_observation {
  data: 0.15968164801597595
  data: 0.6550961136817932
  data: 0.5685981512069702
  data: 0.08132286369800568
  data: 0.7154539823532104
}
reward: 0.5849597454071045
done: true

In [116]:
env_state = EnvStepOutput()

a = env_state.agent_states['0'].data.add()
a.next_observation.data.extend(np.random.rand(5))
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0, 1))

a = env_state.agent_states['1'].data.add()
a.next_observation.data.extend(np.random.rand(5))
a.reward = np.random.rand(1)[0]
a.done = bool(random.randint(0, 1))

m = env_state.agent_metrics['0'].data.add()
m.steps_per_episode = 1
m.step_count = 2
m.temp_done_counter = 3
m.done_count = 4
m.reward_per_step = 5
m.reward_per_episode = 6
m.reward_total = 7
env_state

agent_states {
  key: "0"
  value {
    data {
      next_observation {
        data: 0.16735170781612396
        data: 0.3676755130290985
        data: 0.596005916595459
        data: 0.856003999710083
        data: 0.21721677482128143
      }
      reward: 0.8632524609565735
    }
  }
}
agent_states {
  key: "1"
  value {
    data {
      next_observation {
        data: 0.6031678318977356
        data: 0.3604581952095032
        data: 0.763153612613678
        data: 0.2960234582424164
        data: 0.334116667509079
      }
      reward: 0.7786300778388977
      done: true
    }
  }
}
agent_metrics {
  key: "0"
  value {
    data {
      steps_per_episode: 1.0
      step_count: 2.0
      temp_done_counter: 3.0
      done_count: 4.0
      reward_per_step: 5.0
      reward_per_episode: 6.0
      reward_total: 7.0
    }
  }
}

In [121]:
env_specs = EnvironmentSpecs()
env_specs.observation_space = 1
env_specs.action_space = 2
env_specs.num_instances = 3
env_specs.num_agents_per_instance = 4

In [142]:
env_specs.SerializeToString()

b'\x08\x01\x10\x02\x18\x03 \x04'

In [135]:
buf = env_specs.ListFields()[0]

In [140]:
buf

(<google.protobuf.pyext._message.FieldDescriptor at 0x118c86c50>, 1)

In [125]:
for k, v in dict(env_specs):
    print(k, v)

TypeError: 'EnvironmentSpecs' object is not iterable

In [143]:
a = {}

In [144]:
a.items()

dict_items([])